# Imports

In [1]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import itertools
import random

# Check Gurobi license

In [2]:
try:

    # Create a new model
    m = gp.Model("2.17")

    # Create variables
    c = np.array([1, 2, 3])
    A = np.array([ [1, 2, 3], [1, 1, 0] ])
    b = np.array([4, 1])
    x_prev = np.array([1,2,3])
    x = m.addMVar(3, lb=0.0)
    m.setObjective(c @ x)
    m.addConstr(A@x == b)

    # Optimize model
    m.optimize()

    for v in m.getVars():
        print('%s %g' % (v.VarName, v.X))

    print('Obj: %g' % m.ObjVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-05-11
Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2 rows, 3 columns and 5 nonzeros
Model fingerprint: 0x1990d0aa
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [1e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 2 rows and 3 columns
Presolve time: 0.00s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+00   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Optimal objective  4.000000000e+00
C0 1
C1 0
C2 1
Obj: 4


# Functions

In [2]:
def min_Q_(c: dict, W: dict, h: dict, T: dict, G: dict, g: dict, x_prev: dict, k: int, t: int) -> tuple:

    """
    This function minimizes the function `c[t][k] @ x_t + phi` subject to the following constraints:

    * `W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev`
    * `- G[t+1] @ x_t + phi >= g[t+1]`

    Parameters TODO: Change back to dicts
    ----------
    c : numpy.ndarray
        A 2D array of cost coefficients.
    W : numpy.ndarray
        A 2D array of transition matrices.
    h : numpy.ndarray
        A 1D array of holding costs.
    T : numpy.ndarray
        A 2D array of transfer matrices.
    G : numpy.ndarray
        A 2D array of constraint matrices.
    g : numpy.ndarray
        A 1D array of constraint bounds.
    x_prev : numpy.ndarray
        A 1D array of previous state values.
    k : int
        A scenario index.
    t : int
        A time index.

    Returns
    -------
    x_t_val : numpy.ndarray
        A 1D array of optimal state values.
    obj_val : float
        The optimal objective value.
    pi : float
        The dual value of the first constraint.
    rho : float
        The dual value of the second constraint.
    """

    m = gp.Model("Q_")
    
    x_t = m.addMVar(shape = len(c[t][k]))#, lb=0.0)
    phi = m.addVar()
    
    m.setObjective(c[t][k] @ x_t + phi)
    
    sequence_constraint = m.addConstr(W[t][k] @ x_t == h[t][k] - T[t-1][k] @ x_prev, name = "sequence_constraint")
    cut_constraint = m.addConstr(- G[t+1] @ x_t + phi >= g[t+1] , name = "cut_constraint") #*np.ones_like(g[t+1])
    #print(G[t+1])
    #print(g[t+1])
    m.update()
    #print(f'RHS: {cut_constraint.getAttr("RHS")}')
        
    # Solve
    m.setParam( 'OutputFlag', False )
    m.optimize()
    
    # Primal and objective value
    x_t_val = x_t.getAttr("x")
    obj_val = m.getAttr("ObjVal")
    
    # Duals
    pi = sequence_constraint.getAttr("Pi")
    rho = cut_constraint.getAttr("Pi")

    # Return results
    return x_t_val, obj_val, pi, rho


In [10]:
# Testing
t = 1
k = 0
c = {1: {}}
W = {1: {}}
h = {1: {}}
T = {0: {}}
G = {}
g = {}
c[1][0] = np.array([1, 2, 3])
W[1][0] = np.array([ [1, 2, -4], [1, 1, 0] ])
h[1][0] = np.array([4, 1])
T[0][0] = np.array([ [2,1, 3], [1, 1, 0] ])
x_prev = np.array([1,0,1])
G[2] = np.zeros(shape = (1, len(c[t][0])))
g[2] = np.array([0])
x,Q,pi,rho = min_Q_(c,W,h,T, G,g, x_prev,k,t)

In [11]:
i = 1
t = 1
h = {1: {}}
T = {0: {}}
G = {}
g = {}
q = {}
p = {}
pi = {1:{}}
rho = {1:{}}
c[1][0] = np.array([1, 2, 3])
W[1][0] = np.array([ [1, 2, -4], [1, 1, 0] ])
h[1][0] = np.array([4, 1])
h[1][1] = np.array([2, 1])
h[1][2] = np.array([1, 1])
h[1][3] = np.array([1, 1])
T[0][0] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][1] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][2] = np.array([ [2,1, 3], [1, 1, 0] ])
T[0][3] = np.array([ [2,1, 3], [1, 1, 0] ])
x_prev = np.array([1,0,1])
G[1] = np.zeros(shape = (1, len(c[t][0])))
g[1] = np.array([0])
g[2] = np.array([1])

q[1] = 4
p[1] = np.array([0.1,0.2,0.3,0.4])
pi[1][1] = np.array([ [1,1], [1,3], [2,3], [1,2]])
rho[1][1] = np.array([2,3,4,5])

beta = - sum(p[t][j]*np.matmul(pi[i][t][j],T[t-1][j]) for j in range(q[t]))
print(beta)
alpha = sum(p[t][j] * (np.matmul(pi[i][t][j],h[t][j]) + rho[i][t][j]*g[t+1]) for j in range(q[t]))
print(alpha)
print(G)
print(g)
G[t] = np.vstack((G[t],beta))
g[t] = np.vstack((g[t],alpha))
print(G)
print(g)
#x,Q,pi,rho = min_Q_(c,W,h,T, G,g, x_prev,k,t)

[-5.  -3.7 -3.9]
[8.2]
{1: array([[0., 0., 0.]])}
{1: array([0]), 2: array([1])}
{1: array([[ 0. ,  0. ,  0. ],
       [-5. , -3.7, -3.9]])}
{1: array([[0. ],
       [8.2]]), 2: array([1])}


In [3]:
def calculate_new_cut(q: dict, p: dict, h: dict[int, dict], T: dict[int, dict], 
                      pi: dict[int, dict], rho: dict[int, dict], g: dict, 
                      i: int, t: int, scenario: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Calculates the new cut, i.e. beta and alpha based on the input dicts and dual variables.
    
    Args:
    - q: A dictionary representing the number of elements for each time period.
    - p: A dictionary representing the probabilities of choosing an element for each time period.
    - h: A dictionary of dictionaries representing the values for each element at each time period.
    - T: A dictionary of dictionaries representing the transition matrix for each element at each time period.
    - pi: A dictionary of dictionaries of dictionaries representing the transition probabilities of each element at each time period.
    - rho: A dictionary of dictionaries of dictionaries representing the values of each constraint at each time period.
    - g: A dictionary representing the objective function values for each time period.
    
    Returns:
    A tuple containing the values of beta and alpha.
    """
    beta = - sum(p[t][j] * np.matmul(pi[i][t][scenario][j],T[t-1][j]) for j in range(q[t]))
    alpha = sum(p[t][j] * (np.matmul(pi[i][t][scenario][j],h[t][j]) + np.dot(rho[i][t][scenario][j],g[t+1])) for j in range(q[t]))
    return beta, alpha

In [4]:
def add_cut(G_t: np.array, g_t: np.array, beta: np.array, alpha: np.array) -> tuple[np.array, np.array]:
    assert G_t.shape[0] == g_t.shape[0]
    
    G_t = np.vstack((G_t,beta))
    g_t = np.hstack((g_t,alpha))

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)
    G_t = hstacked_unique[:, :-1]
    g_t = hstacked_unique[:, -1]
    return G_t, g_t


In [5]:
def remove_cut(G_t: np.ndarray, g_t: np.ndarray, beta: np.ndarray, alpha: np.ndarray) -> tuple[np.ndarray, np.ndarray]:

    assert G_t.shape[0] == g_t.shape[0]

    hstacked = np.hstack((G_t,g_t[:,None]))
    hstacked_unique = np.unique(hstacked, axis=0)

    plane = np.hstack((beta, alpha))
    hstacked_removed = hstacked_unique[(hstacked_unique != plane).any(axis=1)]

    G_t = hstacked_removed[:, :-1]
    g_t = hstacked_removed[:, -1]
    return G_t, g_t

In [6]:
G_t = np.array([[0,1,1],[1,1,0]])
g_t = np.array([2,3])
beta = np.array([0,1,1])
alpha = np.array([2])
G_t2, g_t2 = remove_cut(G_t, g_t, beta, alpha)
G_t2, g_t2

(array([[1, 1, 0]]), array([3]))

In [7]:
def calculate_v_upper(c: dict,x_i: dict, sampled_scenarios: list, S: dict, t_max: int) -> float:

    """
    Calculates the upper bound extimate for the value v
    """
    v_upper_i = sum( sum( np.dot(c[t][S[scenario][t]], x_i[t][scenario]) for t in range(1, t_max+1) ) for scenario in sampled_scenarios )
    v_upper_i = v_upper_i / len(sampled_scenarios)
    return v_upper_i


# SDDP

In [25]:
def SDDP(t_max: int, i_max: int, S: dict, c: dict, W: dict, h: dict, T: dict, phi_: np.array, no_of_samples: int, x_0: np.array) -> tuple:
    i = 0
    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    G = {}
    g = {}
    
    v_ = {}
    
    # Initialize cut approximations
    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])        
    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])
    
    
    while i < i_max:
        # Initialize x, Q
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
            
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
        
        # Forward pass
        print(f"Forward pass: {i}")
        sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
        for t in range(1,t_max+1):
            for scenario in sampled_scenarios:
                #print(f"t: {t}, scenario: {scenario}")
                k = S[scenario][t]
                x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t) # Seems good
        
        # calculate upper bound + variances (TODO)
        
        
        # Backward pass
        print(f"Backward pass: {i}")
        for t in reversed(range(1,t_max+1)):
            for scenario in sampled_scenarios:
                
                for j in range(q[t]):
                    #print(f"t: {t}, scenario: {scenario}, j: {j}")
                    x[i][t][scenario], Q_[i][t][scenario], pi[i][t][j], rho[i][t][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t) # j seems good
                    #print(f"rho[{i}][{t}][{j}]: {rho[i][t][j]}")
                
                # Calculate new cut (beta, alpha)
                beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                #print(f"g[{t+1}]: {g[t+1]}")
                #print(f"beta: {beta}")
                #print(f"alpha: {alpha}")
                
                # Add new cut (beta, alpha) to G, g:
                if t != 1:
                    G[t] = np.vstack((G[t],beta))
                    g[t] = np.hstack((g[t],alpha))
                    #print(f"g[{t}]: {g[t]}")
                    
        v_[i] = Q_[i][1][scenario]
        print(f"v_[{i}]: {v_[i]}")
        i = i + 1
    return x, Q_

# BL-SDDP

In [89]:
def BL_SDDP(t_max: int, i_max: int, z_max: int, S: dict[int, tuple[int]], c: dict, W: dict, h: dict, T: dict, 
            phi_: np.array, no_of_samples: int, batch_size: int, x_0: np.array) -> tuple:
    
    assert batch_size <= no_of_samples, "Batch size larger than number of samples per stage"


    x = {}
    Q_ = {}
    pi = {}
    rho = {}
    
    G = {}
    g = {}
    
    v_lower = {}
    v_upper = {}

    memory = {}

    sampled_scenarios = {}
    
    # Initialize cut approximations
    for t in range(2,t_max+1):
        G[t] = np.zeros(shape = (1, len(c[t-1][0])))
        g[t] = np.array([phi_[t]])        
    G[t_max+1] = np.zeros(shape = (1, len(c[t_max][0])))
    g[t_max+1] = np.array([0])

    print('cut approximations initialized')

     # Initialize dicts
    for i in range(0, i_max):
        x[i] = {}
        Q_[i] = {}
        pi[i] = {}
        rho[i] = {}
        
        x[i][0] = {}
        for scenario in list(S.keys()):
            x[i][0][scenario] = x_0
        
        for t in range(1,t_max+1):
            x[i][t] = {}
            Q_[i][t] = {}
            pi[i][t] = {}
            rho[i][t] = {}
            for scenario in list(S.keys()):
                pi[i][t][scenario] = {}
                rho[i][t][scenario] = {}
            

        for t in range(1,t_max+1):
            memory[t] = {}

    print(f'dicts initialized, {len(list(S.keys()))} scenarios')
    
    i = 1

    while i < i_max:
        print(f'Iteration {i}:')

        z = 0
        while z < z_max:


            # Sample no_of_samples scenarios
            sampled_scenarios = random.sample(list(S.keys()), no_of_samples)
            print(f"Sampled scenarios: {sampled_scenarios}")
            
            ## FORWARD PASS

            # Compute actions (forward)
            for t in range(1,t_max+1):
                for scenario in sampled_scenarios:
                    
                    k = S[scenario][t]
                    x[i][t][scenario], Q_[i][t][scenario], pi_temp, rho_temp = min_Q_(c,W,h,T, G,g, x[i][t-1][scenario],k,t)
        
                    # Add trial action to memory
                    memory[t][(i,scenario)] = {"action": x[i][t][scenario]}


            # Calculate upper bound estimate for v
            v_upper[i] = calculate_v_upper(c, x[i], sampled_scenarios, S, t_max)
            print(f"v upper {i}: {v_upper[i]}")


            ## BACKWARD PASS
            
            for t in reversed(range(1,t_max+1)):
                for scenario in sampled_scenarios:
                    
                    # Compute duals (pi, rho) for the scenario
                    for j in range(q[t]):
                    
                        x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i][t-1][scenario],j,t)
                    
                    if t != 1:
                        # Calculate new cut (beta, alpha)
                        beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)
  
                        # Add new cut (beta, alpha) to G, g:
                    
                        G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                        # Store cut
                        memory[t][(i,scenario)]["beta"] = beta
                        memory[t][(i,scenario)]["alpha"] = alpha

            # Increase z
            z = z + no_of_samples
            
        # Select batch keys
        batch_keys = {}
        for t in range(2, t_max+1):

            # Select batch_size number of keys from the memory for stage t
            batch_keys[t] = random.sample(list(memory[t]), batch_size)

            # Remove the K cuts belonging to the K selected actions
            for batch_key in batch_keys[t]:
                beta = memory[t][batch_key]["beta"]
                alpha = memory[t][batch_key]["alpha"]
                if t != 1:
                    G[t], g[t] = remove_cut(G[t], g[t], beta, alpha)
        

        # Backward pass around the K trial actions
        for t in reversed(range(2,t_max+1)):
            for batch_key in batch_keys[t]:
                
                # Get scenario and iteration of the original
                i_batch, scenario = batch_key

                # Compute duals (pi, rho) for the scenario
                for j in range(q[t]):
                    # print((t, batch_key, i, i_batch))
                    # print(x[i_batch][t-1][scenario])
                    x_temp, Q_temp, pi[i][t][scenario][j], rho[i][t][scenario][j] = min_Q_(c,W,h,T,G,g,x[i_batch][t-1][scenario],j,t)
                
                # Calculate new cut (beta, alpha)
                beta, alpha = calculate_new_cut(q,p,h,T,pi,rho,g,i,t,scenario)

                # Add new cut (beta, alpha) to G, g:
                if t != 1:
                    G[t], g[t] = add_cut(G[t], g[t], beta, alpha)

                # Store new cut in memory (exactly where the removed cut was taken from)
                memory[t][batch_key]["beta"] = beta
                memory[t][batch_key]["alpha"] = alpha

        # Finally solve stage t = 1 to get v_lower
        t = 1
        x[i][t][0], Q_[i][t][0], pi_temp, rho_temp = min_Q_(c,W,h,T,G,g,x[i][t-1][0],0,t)

        v_lower[i] = Q_[i][t][0]
        print(f"v lower {i}: {v_lower[i]}")
            
        i = i + 1

    return x, Q_, v_lower, v_upper





## Test 1

In [47]:
## INPUT
t_max = 12

# Uncertainty
q = {t: 2 for t in range(1,t_max+1)}
p = {t: np.array([0.45,0.55]) for t in range(1,t_max+1)}

list_q = [range(q[t]) for t in range(1,t_max+1)]
scenarios_raw = [w for w in itertools.product(range(1),*list_q)]
S = {i: scenarios_raw[i] for i in range(len(scenarios_raw))}

# Parameters
from_grid_max = 25
to_grid_max = 25
soc_max = 100

price_from_grid = {t: {} for t in range(1,t_max+1)}
price_to_grid = {t: {} for t in range(1,t_max+1)}
gen_less_demand = {t: {} for t in range(1,t_max+1)}

for t in range(1, 6):
    price_from_grid[t] = {0: 0.2, 1: 0.4}
    price_to_grid[t] = {0: 0.1, 1: 0.2} 
    gen_less_demand[t] = {0: 0, 1: 10}

for t in range(6, t_max+1):
    price_from_grid[t] = {0: 0.4, 1: 0.6}
    price_to_grid[t] = {0: 0.2, 1: 0.4}
    gen_less_demand[t] = {0: -20, 1: -5}

# Initial condition
x_0 = np.array([0,0,10,0,0,0])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}
for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}
    for j in range(q[t]):
        c[t][j] = np.array([price_from_grid[t][j], -price_to_grid[t][j], 0, 0, 0, 0])
        W[t][j] = np.array([ [-1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1] ])
        h[t][j] = np.array([gen_less_demand[t][j], from_grid_max, to_grid_max, soc_max])
        T[t-1][j] = np.array([ [0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0] ])


# Algorithm parameters
i_max = 30
z_max = 1
no_of_samples = 100
batch_size = 1
phi_list = [-20 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])


In [27]:
x,Q = SDDP(t_max,i_max,S,c,W,h,T,phi_,no_of_samples, x_0)

Forward pass: 0
Backward pass: 0


KeyError: 602

In [48]:
x, Q_, v_lower, v_upper = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 4096 scenarios
Iteration 1:
Sampled scenarios: [794, 3529, 2600, 2905, 2351, 2888, 2484, 730, 2417, 1560, 1739, 1431, 565, 569, 2483, 1159, 1933, 2603, 3241, 2954, 1865, 3526, 1811, 1990, 963, 2034, 3028, 3511, 1909, 1248, 1306, 1164, 1991, 2923, 903, 2421, 1810, 1258, 801, 2236, 981, 290, 2209, 4067, 3058, 3682, 1645, 393, 2363, 1648, 152, 3652, 145, 1333, 270, 1849, 2617, 2152, 266, 3362, 1313, 3382, 1142, 4013, 2751, 3880, 231, 319, 2255, 3758, 2783, 1916, 1778, 3913, 1203, 1374, 2402, 2342, 2476, 3550, 3381, 567, 3103, 1426, 3978, 158, 210, 1309, 672, 2178, 3409, 311, 594, 2822, 1110, 1850, 950, 359, 2211, 2999]
v upper 1: 32.0
v lower 1: -2.269251090813346
Iteration 2:
Sampled scenarios: [2891, 825, 3609, 1280, 1984, 719, 437, 971, 2972, 3515, 1269, 2520, 3429, 2613, 3315, 3267, 3377, 2919, 2958, 1396, 3168, 1028, 3431, 3633, 1536, 1147, 304, 2963, 1889, 1035, 3705, 3775, 1201, 3462, 3517, 2355, 1358, 1942, 2617, 3899, 4035, 1641, 

In [56]:
v_lower, 
v_upper, 
Q_[29][2] # TODO: find out what is going on

{3808: 8.48251250000001,
 2578: 9.391243750000006,
 1821: 4.391243750000006,
 1243: 4.391243750000006,
 2085: 9.391243750000006,
 2482: 9.391243750000006,
 3856: 8.48251250000001,
 2254: 9.391243750000006,
 3225: 8.48251250000001,
 2059: 9.391243750000006,
 390: 6.150000000000011,
 1255: 4.391243750000006,
 603: 6.150000000000011,
 1388: 4.391243750000006,
 1692: 4.391243750000006,
 3292: 8.48251250000001,
 368: 6.150000000000011,
 1877: 4.391243750000006,
 4026: 8.48251250000001,
 552: 6.150000000000011,
 3853: 8.48251250000001,
 48: 6.150000000000011,
 2741: 9.391243750000006,
 1123: 4.391243750000006,
 2570: 9.391243750000006,
 1466: 4.391243750000006,
 587: 6.150000000000011,
 3043: 9.391243750000006,
 1531: 4.391243750000006,
 1110: 4.391243750000006,
 1419: 4.391243750000006,
 1417: 4.391243750000006,
 2499: 9.391243750000006,
 2745: 9.391243750000006,
 982: 6.150000000000011,
 2177: 9.391243750000006,
 668: 6.150000000000011,
 2503: 9.391243750000006,
 409: 6.150000000000011,
 2

In [143]:
q = {1: 2, 2: 2, 3: 2}
S = [w for w in itertools.product(range(1),range(q[1]),range(q[2]),range(q[3]))]
scenarios = {i: S[i] for i in range(len(S))}
print(scenarios)

[(0, 0, 0, 0), (0, 0, 0, 1), (0, 0, 1, 0), (0, 0, 1, 1), (0, 1, 0, 0), (0, 1, 0, 1), (0, 1, 1, 0), (0, 1, 1, 1)]
{0: (0, 0, 0, 0), 1: (0, 0, 0, 1), 2: (0, 0, 1, 0), 3: (0, 0, 1, 1), 4: (0, 1, 0, 0), 5: (0, 1, 0, 1), 6: (0, 1, 1, 0), 7: (0, 1, 1, 1)}


# Examples

## Toy example

In [121]:
## INPUT
t_max = 3

# Uncertainty
q = {t: 3 for t in range(2,t_max+1)}
q[1] = 1
print(q)

p = {t: np.array([0.3333,0.3334,0.3333]) for t in range(2,t_max+1)}

list_q = [range(q[t]) for t in range(1,t_max+1)]
scenarios_raw = [w for w in itertools.product(*list_q)]
S = {i: {t: scenarios_raw[i][t-1] for t in range(1,len(scenarios_raw[i])+1)} for i in range(len(scenarios_raw))}
print(S)


# Initial condition
x_0 = np.array([ 0, 0 ])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}

for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}

c[1][0] = np.array([ 1, 0 ])
c[2][0] = np.array([ 1, 0 ])
c[2][1] = np.array([ 1, 0 ])
c[2][2] = np.array([ 1, 0 ])
c[3][0] = np.array([ 1, 1 ])
c[3][1] = np.array([ 1, 1 ])
c[3][2] = np.array([ 1, 1 ])

W[1][0] = np.array([ [1, 1] ])
W[2][0] = np.array([ [1, -1] ])
W[2][1] = np.array([ [1, -1] ])
W[2][2] = np.array([ [1, -1] ])
W[3][0] = np.array([ [1, -1] ])
W[3][1] = np.array([ [1, -1] ])
W[3][2] = np.array([ [1, -1] ])

h[1][0] = np.array([ 6 ])
h[2][0] = np.array([ 4 ])
h[2][1] = np.array([ 5 ])
h[2][2] = np.array([ 6 ])
h[3][0] = np.array([ 1 ])
h[3][1] = np.array([ 2 ])
h[3][2] = np.array([ 4 ])

T[0][0] = np.array([ [0, 0] ])
T[1][0] = np.array([ [1, 0] ])
T[1][1] = np.array([ [1, 0] ])
T[1][2] = np.array([ [1, 0] ])
T[2][0] = np.array([ [1, 0] ])
T[2][1] = np.array([ [1, 0] ])
T[2][2] = np.array([ [1, 0] ])


# Algorithm parameters
i_max = 20
z_max = 1
no_of_samples = 6
batch_size = 1
phi_list = [-100 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])
print(phi_)


{2: 3, 3: 3, 1: 1}
{0: {1: 0, 2: 0, 3: 0}, 1: {1: 0, 2: 0, 3: 1}, 2: {1: 0, 2: 0, 3: 2}, 3: {1: 0, 2: 1, 3: 0}, 4: {1: 0, 2: 1, 3: 1}, 5: {1: 0, 2: 1, 3: 2}, 6: {1: 0, 2: 2, 3: 0}, 7: {1: 0, 2: 2, 3: 1}, 8: {1: 0, 2: 2, 3: 2}}
[   0 -100 -100 -100]


In [122]:
x, Q_, v_lower, v_upper = BL_SDDP(t_max=t_max, i_max=i_max, z_max=z_max, 
                                  S=S, c=c, W=W, h=h, T=T, 
                                  phi_ = phi_, no_of_samples = no_of_samples, batch_size = batch_size,
                                  x_0 = x_0)

cut approximations initialized
dicts initialized, 9 scenarios
Iteration 1:
Sampled scenarios: [3, 6, 5, 7, 2, 4]
v upper 1: 8.0
v lower 1: 4.312410945116886
Iteration 2:
Sampled scenarios: [1, 0, 4, 2, 6, 8]
v upper 2: 6.874940630077924
v lower 2: 5.5554444499999995
Iteration 3:
Sampled scenarios: [4, 0, 5, 3, 6, 8]
v upper 3: 6.261894558202087
v lower 3: 5.899813991841071
Iteration 4:
Sampled scenarios: [3, 7, 0, 8, 1, 4]
v upper 4: 7.899813991841071
v lower 4: 6.1332279956804605
Iteration 5:
Sampled scenarios: [1, 7, 8, 5, 0, 3]
v upper 5: 6.2666426652268195
v lower 5: 6.222144439999999
Iteration 6:
Sampled scenarios: [7, 3, 6, 2, 4, 5]
v upper 6: 6.666666666666667
v lower 6: 6.222144439999999
Iteration 7:
Sampled scenarios: [8, 0, 2, 4, 1, 3]
v upper 7: 5.666666666666667
v lower 7: 6.222144439999999
Iteration 8:
Sampled scenarios: [2, 5, 1, 7, 6, 4]
v upper 8: 6.5
v lower 8: 6.222144439999999
Iteration 9:
Sampled scenarios: [2, 8, 3, 6, 4, 0]
v upper 9: 6.166666666666667
v lower 9: 

In [123]:
x[9][1]

{2: array([3., 3.]),
 8: array([3., 3.]),
 3: array([3., 3.]),
 6: array([3., 3.]),
 4: array([3., 3.]),
 0: array([3., 3.])}

## Energy example

In [ ]:
# TODO: Modify input according to specification in toy example, i.e. t=1 deterministic

## INPUT

t_max = 12

# Uncertainty
q = {t: 2 for t in range(1,t_max+1)}
p = {t: np.array([0.45,0.55]) for t in range(1,t_max+1)}

list_q = [range(q[t]) for t in range(1,t_max+1)]
scenarios_raw = [w for w in itertools.product(range(1),*list_q)]
S = {i: scenarios_raw[i] for i in range(len(scenarios_raw))}

# Parameters
from_grid_max = 25
to_grid_max = 25
soc_max = 100

price_from_grid = {t: {} for t in range(1,t_max+1)}
price_to_grid = {t: {} for t in range(1,t_max+1)}
gen_less_demand = {t: {} for t in range(1,t_max+1)}

for t in range(1, 6):
    price_from_grid[t] = {0: 0.2, 1: 0.4}
    price_to_grid[t] = {0: 0.1, 1: 0.2} 
    gen_less_demand[t] = {0: 0, 1: 10}

for t in range(6, t_max+1):
    price_from_grid[t] = {0: 0.4, 1: 0.6}
    price_to_grid[t] = {0: 0.2, 1: 0.4}
    gen_less_demand[t] = {0: -20, 1: -5}

# Initial condition
x_0 = np.array([0,0,10,0,0,0])

# Generate input represenation
c = {}
W = {}
h = {}
T = {}
for t in range(1,t_max+1):
    c[t] = {}
    W[t] = {}
    h[t] = {}
    T[t-1] = {}
    for j in range(q[t]):
        c[t][j] = np.array([price_from_grid[t][j], -price_to_grid[t][j], 0, 0, 0, 0])
        W[t][j] = np.array([ [-1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 0, 1] ])
        h[t][j] = np.array([gen_less_demand[t][j], from_grid_max, to_grid_max, soc_max])
        T[t-1][j] = np.array([ [0, 0, -1, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0] ])


# Algorithm parameters
i_max = 30
z_max = 1
no_of_samples = 100
batch_size = 1
phi_list = [-20 for t in range(1, t_max+1)]
phi_ = np.array([0,*phi_list])
